<img style="float: center;" src="images/CI_horizontal.png" width="600">
<center>
    <span style="font-size: 1.5em;">
        <a href='https://www.coleridgeinitiative.org'>Website</a>
    </span>
</center>

Ghani, Rayid, Frauke Kreuter, Julia Lane, Adrianne Bradford, Alex Engler, Nicolas Guetta Jeanrenaud, Graham Henke, Daniela Hochfellner, Clayton Hunter, Brian Kim, Avishek Kumar, and Jonathan Morgan.

# Dataset Exploration
----------
Basic dataset exploration

## Table of Contents

- [Introduction](#Introduction)
    - [Learning Objectives](#Learning-Objectives)
    - [Methods](#Methods)
- [Python Setup](#Python-Setup)
- [Load the Data](#Load-the-Data)
    - [Establish a Connection to the Database](#Establish-a-Connection-to-the-Database)
    - [Formulate Data Query](#Formulate-Data-Query)
    - [Pull Data from the Database](#Pull-Data-from-the-Database)
- [Analysis: Using Python and SQL to Analyze Economic Activity in KCMO](#Analysis:-Using-Python-and-SQL-to-Analyze-Economic-Activity-in-KCMO)
    - [What is in the Database?](#What-is-in-the-Database?)
    - [Summary Statistics on Different Datasets](#Summary-Statistics-on-Different-Datasets)
    - [Combining Datasets](#Combining-Datasets)
    - [Creating New Measures](#Creating-New-Measures)
- [Exercise](#Exercise)
- [Submit Results](#Submit-Results)

## Introduction
- Back to [Table of Contents](#Table-of-Contents)

In an ideal world, we will have all of the data we want with all of the desirable properties (no missing values, no errors, standard formats, and so on). 
However, that is hardly ever true - and we have to work with using our datasets to answer questions of interest as intelligently as possible. 

In this notebook, we will discover the datasets we have on the ADRF, and we will use our datasets to answer some questions of interest. 

### Learning Objectives
- Back to [Table of Contents](#Table-of-Contents)

This notebook will give you the opportunity to spend some hands-on time with the data. 

You will have an opportunity to explore the different datasets in the ADRF, and this notebook will take you around the different ways you can analyze your data. This involves looking at basic metrics in the larger dataset, taking a random sample, creating derived variables, making sense of the missing values, and so on. 

This will be done using both SQL and `pandas` in Python. The `sqlalchemy` Python package will give you the opportunity to interact with the database using SQL to pull data into Python. Some additional manipulations will be handled by Pandas in Python (by converting your datasets into dataframes).

This notebook will provide an introduction and examples for: 

- How to create new tables from the larger tables in database (sometimes called the "analytical frame")
- How to explore different variables of interest
- How to explore aggregate metrics
- How to handle missing values
- How to join newly created tables

### Methods
- Back to [Table of Contents](#Table-of-Contents)

We will be using the `sqlalchemy` Python package to access tables in our class database server - PostgreSQL. 

To read the results of our queries, we will be using the `pandas` Python package, which has the ability to read tabular data from SQL queries into a pandas DataFrame object. Within `pandas`, we will use various commands:

- Subsetting data
- `groupby`
- `merge`

Within SQL, we will use various queries to:

- select data subsets
- Sum over groups
- create new tables
- Count distinct values of desired variables
- Order data by chosen variables
- Select a random sub-sample

## Python Setup
- Back to [Table of Contents](#Table-of-Contents)

In Python, we `import` packages. The `import` command allows us to use libraries created by others in our own work by "importing" them. You can think of importing a library as opening up a toolbox and pulling out a specific tool. Among the most famous Python packages:
- `numpy` is short for "numerical Python". `numpy` is a lynchpin in Python's scientific computing stack. Its strengths include a powerful *N*-dimensional array object, and a large suite of functions for doing numerical computing. 
- `pandas` is a library in Python for data analysis that uses the DataFrame object (modeled after R DataFrames, for those familiar with that language) which is similiar to a spreedsheet but allows you to do your analysis programaticaly rather than the point-and-click of Excel. It is a lynchpin of the PyData stack and is built on top of `numpy`.  
- `sqlalchemy` is a Python library for interfacing with a PostGreSQL database. 

In [ ]:
# pandas-related imports
import pandas as pd

# database interaction imports
import sqlalchemy

__When in doubt, use shift + tab to read the documentation of a method.__

__The `help()` function provides information on what you can do with a function.__

In [ ]:
# for example
help(sqlalchemy.create_engine)

## Load the Data

- Back to [Table of Contents](#Table-of-Contents)

We can execute SQL queries using Python to get the best of both worlds. For example, Python - and pandas in particular - make it much easier to calculate descriptive statistics of the data. Additionally, as we will see in the Data Visualization exercises, it is relatively easy to create data visualizations using Python. 

Pandas provides many ways to load data. It allows the user to read the data from a local csv or excel file, pull the data from a relational database, or read directly from a URL (when you have internet access). Since we are working with the PostgreSQL database `appliedda` in this course, we will demonstrate how to use pandas to read data from a relational database. For examples to read data from a CSV file, refert to the pandas documentation [Getting Data In/Out](pandas.pydata.org/pandas-docs/stable/10min.html#getting-data-in-out).

The function to run a SQL query and pull the data into a pandas dataframe (more to come) is `pd.read_sql()`. Just like doing a SQL query from pgAdmin, this function will ask for some information about the database, and what query you would like to run. Let's walk through the example below.

### Establish a Connection to the Database
- Back to [Table of Contents](#Table-of-Contents)

The first parameter is the connection to the database. To create a connection we will use the SQLAlchemy package and tell it which database we want to connect to, just like in pgAdmin. Additional details on creating a connection to the database are provided in the [Databases](02_1_Databases.ipynb) notebook.

__Parameter 1: Connection__

In [ ]:
# to create a connection to the database, 
# we need to pass the name of the database and host of the database

host = 'stuffed.adrf.info'
DB = 'appliedda'

connection_string = "postgresql://{}/{}".format(host, DB)
conn = sqlalchemy.create_engine(connection_string)

> Note we can parameterize Python `string` objects - using the built-in `.format()` function. We will use various formulations in the program notebooks (eg when building queries), some examples are:
1. Empty brackets (shown above) which simply inserts the variable in the string; when there is more than one set of brackets Python will insert variables in the order they are listed
2. Brackets with formatting can be used to make print statements more readable (eg `'text with formatted number with comma and 1-digit decimal {:,.1f}'.format(number_value)` will print `123,456.7` instead of `123456.7123401`)
3. Named brackets to use the same variables multiple times in a text block (we use this in more compicated queries eg when creating "labels" and "features" for Machine Learning models)

### Formulate Data Query
- Back to [Table of Contents](#Table-of-Contents)

This part is similar to writing a SQL query in pgAdmin. Depending on what data we are interested in, we can use different queries to pull different data. In this example, we will pull all the content of the IL QCEW employers data.

__create a query as a `string` object in Python__

In [ ]:
query = '''
SELECT *
FROM il_des_kcmo.il_qcew_employers
WHERE year = 2014 AND quarter = 2
LIMIT 20
'''

Note:

- the three quotation marks surrounding the query body is called multi-line string. It is quite handy for writing SQL queries because the new line character will be considered part of the string, instead of breaking the string

In [ ]:
# Now that we have defined a variable `query`, we can call it in the code
print(query)

> Note that the `LIMIT` provides one simple way to get a "sample" of data; however, using `LIMIT` does **not provide a _random_** sample. You may get different samples of data than others using just the `LIMIT` clause, but it is just based on what is fastest for the database to return.

### Pull Data from the Database
- Back to [Table of Contents](#Table-of-Contents)

Now that we have the two parameters (database connection and query), we can pass them to the `pd.read_sql()` function, and obtain the data.

In [ ]:
# here we pass the query and the connection to the pd.read_sql() function and assign the variable `wage` 
# to the dataframe returned by the function
df = pd.read_sql(query, conn)

In [ ]:
df.head()

## Analysis: Using Python and SQL
- Back to [Table of Contents](#Table-of-Contents)

__What are different measures of employment for TANF recipients?__

To explore possible metrics, we will need to combine TANF and employment (in our case, UI wage records) data. We will start slow and explore these datasets individually, then work up to some initial metrics of employment before, during and after receipt of TANF benefits.

### What is in the Database?
- Back to [Table of Contents](#Table-of-Contents)

As introduced in the [Databases](./02_1_Databases.ipynb) notebook, there are many ways to connect

__ Schemas, Tables, and Columns in database__

Let's pull the list of schema names in the database, the list of tables in these schemas and the list of columns in these tables.

In [ ]:
# See all available schemas:
query = '''
SELECT schema_name 
FROM information_schema.schemata;
'''
pd.read_sql(query, conn)

> As a reminder, in this class you have access to the following schemas: 'public', 'il_des_kcmo', 'il_dhs', 'il_doc_kcmo', 'kcmo_lehd', 'ada_tanf' and your team schema ('ada_tanf_#', where the # is your team number)

In [ ]:
schemas = """
'public', 'il_des_kcmo', 'il_dhs', 'il_doc_kcmo', 'kcmo_lehd',  'ada_tanf'
"""

In [ ]:
# confirm our schemas exist with 
# an updated version of the previous query
query = '''
SELECT schema_name 
FROM information_schema.schemata
WHERE schema_name IN ({})
'''.format(schemas)
pd.read_sql(query, conn)

In [ ]:
query = '''
SELECT schemaname, tablename
FROM pg_tables
WHERE schemaname IN ({})
'''.format(schemas)

tables = pd.read_sql(query, conn)
# print tables not in the public schema
print(tables.query("schemaname != 'public'"))

In [ ]:
# list all the tables in the IL DHS schema:
sorted(tables[tables["schemaname"] == 'il_dhs']['tablename'])

> Note the two ways shown above to subset a `Pandas.DataFrame`:
1. Use the built-in `.query()` function
2. Create an array of `True` and `False` values (done in this line: `tables["schemaname"] == 'il_dhs'`)

In [ ]:
# We can look at column names within tables
# here we'll set the schema and table with variables

schema = 'il_dhs'
tbl = 'ind_spells'

query = '''
SELECT * 
FROM information_schema.columns 
WHERE table_schema = '{}' AND table_name = '{}'
'''.format(schema, tbl)

# read and print results
pd.read_sql(query, conn)

__Illinois DES datasets__:
- `il_des_kcmo.il_qcew_employers`: business level data
- `il_des_kcmo.il_wage`: individual job level data
- `il_des_kcmo.il_des_establishment`: improved location data for establishements (includes years 2012-2015)

In [ ]:
query = '''
SELECT *
FROM il_des_kcmo.il_qcew_employers
limit 100;
'''
il_qcew_employers = pd.read_sql(query, conn)

In [ ]:
il_qcew_employers.head()

Again, take some time to look at the documentation and understand what the different variables refer to.

Some employer NAICS codes seem to be missing. Let's see how many records have a value in the `naics` column.

In [ ]:
# set the SQL query
query ="""
SELECT count(*) 
FROM il_des_kcmo.il_qcew_employers 

WHERE naics IS NOT NULL

AND year = 2009 AND quarter = 2;
"""
# read the query into a DataFrame
has_naics = pd.read_sql(query, conn)

# print the resulting DataFrame
has_naics

In [ ]:
# here's how we can access the value in our 'has_naics' DataFrame
has_naics['count'][0]

In [ ]:
# What percentage of the records have a NAICS code?

# set the SQL query
query ="""
SELECT {}./count(*) AS not_missing
FROM il_des_kcmo.il_qcew_employers
WHERE year = 2009 
    AND quarter = 2;
""".format(has_naics['count'][0])

# print the query for reference
print(query)

# read the query and print the result

print('{:.1f}% of records have a NAICS code'\
.format(pd.read_sql(query, conn)['not_missing'][0]*100))

Also, some employers are missing their legal name. Let's see how many.

In [ ]:
# It is likely that you will see that some employers do not have a legal name. 
# Let's find how many.

#generating read SQL
query = '''
SELECT count(distinct ein)
FROM il_des_kcmo.il_qcew_employers
WHERE name_legal is NULL
AND year = 2009 AND quarter = 2
'''
# read the query into a DataFrame
missing_names = pd.read_sql(query, conn)
# print the resulting DataFrame
missing_names

> **Discuss with your team:** what we should do about these missing values?

**Exercise:** try coding a similar simple exploration of the `il_des_establishment` record table as above

In [ ]:
#### your code...


## Summary Statistics
- Back to [Table of Contents](#Table-of-Contents)

In this section, let's start looking at aggregate statistics on the data. 

Let's explore a few specific, simple questions to better understand our data:
- How many jobs are there in our data? 
- What wages are paid for different types of jobs?
- How many TANF cases are there? How many individuals receive TANF?
- How many TANF recipients have a job before, during, and after they are enrolled in TANF benefits?

> Note: __ Large tables__ can take a long time to process on shared databases. The IL DES UI wage data has over 6M records per quarter, so we will demonstrate using SQL and Python with consideration for how much data we are reading back into Python

In [ ]:
# additionally we'll use the Python time package
# to see how long different queries takes to return

import time # import time package

In [ ]:
# example count of records for 2007 Q2 

start_time = time.time() # get current time

qry = """
SELECT count(*) 
FROM il_des_kcmo.il_wage         
WHERE year = 2007 AND quarter = 2
"""
# print results
print(pd.read_sql(qry, conn)) 
# print analysis time
print('Query returned in {:.1f} seconds'.format(time.time()-start_time))

> A **question to consider**: This simple count is one measure of the total jobs in IL in 2007 Q2. What may we want to consider when defining a "job" in addition to just being a row in this dataset?

In [ ]:
# the easiest way to look at a small subset of records is what
# is already demonstrated above: simply add a LIMIT clause
# again we'll look at how long this query takes to return

# get current time
start_time = time.time()

query = '''
SELECT *
FROM il_des_kcmo.il_wage
WHERE year = 2007 AND quarter = 2
LIMIT 20;
'''
# get results
df_earnings = pd.read_sql(query, conn)
# print analysis time
print('Query returned in {:.1f} seconds'.format(time.time()-start_time))

In [ ]:
df_earnings.head()

In [ ]:
df_earnings.columns

> Reminder: you can refer to the documentation for more information on each dataset

In [ ]:
# we can get descriptive stats from the DataFrame:
df_earnings.describe(include='all')

In [ ]:
# as with the 'count` example, we will get basic stats
# from the database using SQL; eg the wage distribution

# first, just the 25th percentile value
query = """
SELECT percentile_cont(0.25)
    WITHIN GROUP (ORDER BY wage)
FROM il_des_kcmo.il_wage
WHERE year = 2007 AND quarter = 2
"""
# display result
pd.read_sql(query, conn)

> Note: as of Jan 1, 2016, the minimum wage in IL was \$8.25 per hour. Assuming a 35 hour work-week and 12 weeks in a quarter, someone working an entire quarter at minumum wage would earn \$3,465 in the quarter (ignoring taxes). In 2007, IL minimum wage was \$6.50 per hour, or \$2,730 per quarter (with the same assumptions).

In [ ]:
# use a list of percentile values at which to show the earnings value
query = """
SELECT percentile_cont(array[0.1, 0.25, 0.5, 0.75, 0.9])
WITHIN GROUP (ORDER BY wage)
FROM il_des_kcmo.il_wage
WHERE year = 2007 AND quarter = 2
"""
# display result
pd.read_sql(query, conn)

In [ ]:
# values above return in a single cell, 
# add "unnest" to get values in a single cell
# also add a reference column and column names

start_time = time.time()

query = """
SELECT unnest(
        percentile_cont(array[0.1, 0.25, 0.5, 0.75, 0.9])
        WITHIN GROUP (ORDER BY wage)
    ) AS earnings_value,
    unnest(array[0.1, 0.25, 0.5, 0.75, 0.9]) AS percentile_value
FROM il_des_kcmo.il_wage
WHERE year = 2007 AND quarter = 2
"""
# get the result
df = pd.read_sql(query, conn)
# print analysis time
print('Query returned in {:.1f} seconds'.format(time.time()-start_time))
# view result
df

One way to characterize a job is the employer industry. In order to run summary statistics on number of jobs per industry (NAICS code), we need to get the NAICS code from the `il_qcew_employers` table

In [ ]:
# check how many records from our Earnings data matches the Employers data
# just for 2007 Q2 data

start_time = time.time()

qry = """
SELECT count(*) 
FROM il_des_kcmo.il_wage AS earn
JOIN il_des_kcmo.il_qcew_employers AS empl
ON earn.ein = empl.ein 
    AND earn.seinunit = empl.seinunit
    AND earn.empr_no = empl.empr_no
WHERE earn.year = 2007 AND earn.quarter = 2 
    AND empl.year = 2007 AND empl.quarter = 2
"""
res = pd.read_sql(qry, conn)
print('query took {:.1f} seconds'.format(time.time()-start_time))

In [ ]:
res

### Query time and the PostgreSQL EXPLAIN

As you begin creating more advanced or complicated queries, it is useful to have a sense of how long different queries take. One way is to simple record and observe how long different queries take, as has been done in the notebook so far.

You may get a sense of when a query is taking a long time. Slow queries could be the result of many people using the database at the same time, the database or table you are using are being `VACUUM`ed, or because of a poorly formed query or data structure (eg not making use of or having `indexes` on columns used to frequently subset or match the data)

> **The PostgreSQL "Explain" function** can help determine in what order the query is executed and whether it is making use of indexes. It is a little difficult to interpret, but here is an exmaple using the above query. The way to read this is start at the bottom of the RESULT and read up (steps outlined below the query output)
>   
    -- QUERY:
    EXPLAIN 
    SELECT count(*) 
    FROM il_des_kcmo.il_wage AS earn
    JOIN il_des_kcmo.il_qcew_employers AS empl
    ON earn.ein = empl.ein AND earn.seinunit = empl.seinunit
        AND earn.empr_no = empl.empr_no
    WHERE earn.year = 2007 AND earn.quarter = 2 
    AND empl.year = 2007 AND empl.quarter = 2
>  
    -- RESULT
    Aggregate  (cost=2733713.11..2733713.12 rows=1 width=0)
    ->  Merge Join  (cost=2659260.50..2733713.11 rows=1 width=0)
         Merge Cond: ((earn.ein = empl.ein) AND (earn.seinunit = empl.seinunit) AND (earn.empr_no = empl.empr_no))
         ->  Sort  (cost=1563641.97..1580871.12 rows=6891659 width=82)
               Sort Key: earn.ein, earn.seinunit, earn.empr_no
               ->  Append  (cost=0.00..451095.87 rows=6891659 width=82)
                     ->  Seq Scan on il_wage earn  (cost=0.00..0.00 rows=1 width=96)
                           Filter: ((year = 2007) AND (quarter = 2))
                     ->  Seq Scan on il_wage_2007q2 earn_1  (cost=0.00..451095.87 rows=6891658 width=82)
                           Filter: ((year = 2007) AND (quarter = 2))
         ->  Materialize  (cost=1095618.53..1097832.93 rows=442881 width=80)
               ->  Sort  (cost=1095618.53..1096725.73 rows=442881 width=80)
                     Sort Key: empl.ein, empl.seinunit, empl.empr_no
                     ->  Bitmap Heap Scan on il_qcew_employers empl  (cost=13816.09..1034403.41 rows=442881 width=80)
                           Recheck Cond: ((year = 2007) AND (quarter = 2))
                           ->  Bitmap Index Scan on il_des_kcmo_qcew_employers_year_quarter_index  (cost=0.00..13705.37 rows=442881 width=0)
                                 Index Cond: ((year = 2007) AND (quarter = 2))

So, this query runs in the following steps:
1. scans the employer year and quarter indexes
2. returns only rows where year and quarter meet our criteria
3. sorts the result based on our 3 identifier columns
4. performs same 3 steps on the earnings data (scan, filter, sort)
5. merges the two tables based on the 3 identifier columns
6. counts the rows (the final, "Aggregate" step)

For example, try running explain on a slightly different variant of the previous query:

    EXPLAIN SELECT count(*) 
    FROM il_des_kcmo.il_wage AS earn
    JOIN il_des_kcmo.il_qcew_employers AS empl
    ON earn.ein = empl.ein 
        AND earn.seinunit = empl.seinunit
        AND earn.empr_no = empl.empr_no
        AND earn.year = empl.year
        AND earn.quarter = empl.quarter

In [ ]:
# let's peruse a sample of a combined dataset

start_time = time.time()

qry = """
SELECT earn.ssn, earn.ein, earn.seinunit, earn.empr_no, earn.wage AS earnings,
    empl.name_legal, empl.name_trade, multi_unit_code, naics, auxiliary_naics, 
    total_wages AS tot_wages_paid
FROM il_des_kcmo.il_wage AS earn
JOIN il_des_kcmo.il_qcew_employers AS empl
ON earn.ein = empl.ein AND earn.seinunit = empl.seinunit
    AND earn.empr_no = empl.empr_no
WHERE earn.year = 2007 AND earn.quarter = 2 
AND empl.year = 2007 AND empl.quarter = 2
LIMIT 100
"""
res = pd.read_sql(qry, conn)

# report how long it took to pull data
print('query took {:.1f} seconds'.format(time.time()-start_time))

In [ ]:
res.head()

In [ ]:
res.describe()

In [ ]:
# what businesses are in our sample of 100 jobs? and how many jobs are associated with each?
res['name_trade'].value_counts()

In [ ]:
# and industries?
res['naics'].value_counts()

> As we see here, the data have 6 digit NAICS codes. We will revisit this when we look at the breakdown of jobs and earnings by industries.

In [ ]:
# what is the distribution of earnings in our sample?
res['earnings'].describe(percentiles=[0.1,0.25,0.5, 0.75, 0.9])

In [ ]:
# and earnings by industry?
res.groupby('naics')['earnings'].describe(percentiles=[0.1,0.25,0.5, 0.75, 0.9])

In [ ]:
# now we'll query the database for earnings distribution by 2-digit NAICS in Q2 2007

start_time = time.time()

query="""
SELECT naics2, 
    unnest(percentile_cont(array[0.1,0.25,0.5, 0.75, 0.9]) 
    within group (ORDER BY earnings)) AS earnings,
    unnest(array[0.1,0.25,0.5, 0.75, 0.9]) AS percentiles
FROM (
    SELECT earn.wage AS earnings, left(empl.naics, 2) AS naics2
    FROM il_des_kcmo.il_wage AS earn
    JOIN il_des_kcmo.il_qcew_employers AS empl
    ON earn.ein = empl.ein AND earn.seinunit = empl.seinunit
        AND earn.empr_no = empl.empr_no
    WHERE earn.year = 2007 AND earn.quarter = 2 
    AND empl.year = 2007 AND empl.quarter = 2
) subquery
GROUP BY naics2
ORDER BY naics2, percentiles
"""
res = pd.read_sql(query, conn)
print('query took {:.1f} seconds'.format(time.time()-start_time))

In [ ]:
res.info()

In [ ]:
# we can see from the count of non-null values in the `naics2` column that 
# there are some missing values in the data - we'll insert an "Unknown" flag:
res.naics2.fillna('Unknown', inplace=True)

In [ ]:
# DataFrames also have useful functions like pivot tables:
res.pivot_table(values='earnings', columns='percentiles', index='naics2')

> Wages paid by different industry is one way to explore how the **jobs very across industry groups.**

## Exploring TANF data

Our questions:
- How many individuals receive TANF? What are their characteristics?
- How many TANF recipients have a job before, during, and after they are enrolled in TANF benefits?

Since the TANF data have start and end dates, we will need to consider how our questions relate to dates (whereas with the wage record data we only know if people were paid during a given quarter).

Additionally, the TANF data are much more complex so here will focus on just two tables:
1. `ind_spells` - individual level spells on different benefits (we'll further focus on just the TANF data, coded as 'tanf46' in this data)
2. `member` - includes more information about the people, such as birthdate and gender

In [ ]:
# How many spells end in Q4 of 2006?

start_time = time.time()

query="""
SELECT count(*) 
FROM il_dhs.ind_spells
WHERE end_date >= '2006-10-01'::date AND 
    end_date < '2007-01-01'::date
    AND benefit_type = 'tanf46'
"""

print('there are {:,.0f} TANF spells'.format(pd.read_sql(query, conn)['count'][0]))
print('query completed in {:.2f} seconds'.format(time.time()-start_time))

In [ ]:
# let's pull the list's information in the member table
# in this query we grab just the recptno values for our
# cohort, then use that list to pull the info from "member"
start_time = time.time()
query = """
SELECT * FROM il_dhs.member
WHERE recptno IN ( SELECT recptno
    FROM il_dhs.ind_spells
    WHERE end_date >= '2006-10-01'::date AND 
        end_date < '2007-01-01'::date
        AND benefit_type = 'tanf46')
"""
df = pd.read_sql(query, conn)
print('query completed in {:.2f} seconds'.format(time.time()-start_time))

In [ ]:
df.info()

In [ ]:
# there are many more results than just our list
# does the unique list of recptno match?
df['recptno'].nunique()

In [ ]:
# how many unque values for the 'static' variables?
static_vars = ['recptno', 'ssn_hash', 'sex', 'rac', 'rootrace', 'foreignbn', 'birth_date']

df[static_vars].nunique()

In [ ]:
df.groupby(static_vars)['update_id'].count()\
.reset_index()\
.rename(columns={'update_id': 'records'})\
.sort_values('records',ascending=False).head()

In [ ]:
df.groupby(static_vars)['update_id'].count().shape

Instead of digging through how the member table is constructed, let's instead base our cohort selection on the `ind_spells` table

In [ ]:
# columns from the member table
print(df.columns.tolist())

In [ ]:
start_time = time.time()
query = """
SELECT DISTINCT ON (i.recptno) i.recptno, i.start_date, i.end_date, 
    m.birth_date, m.ssn_hash, sex, rac, rootrace
FROM il_dhs.ind_spells i
LEFT JOIN il_dhs.member m
ON i.recptno = m.recptno
WHERE end_date >= '2006-10-01'::date AND 
        end_date < '2007-01-01'::date
        AND benefit_type = 'tanf46'
"""
# read the data, and additionally parse the dates
df = pd.read_sql(query, conn, parse_dates=['start_date', 'end_date', 'birth_date'])
print('data read in {:.2f} seconds'.format(time.time()-start_time))

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
# how many of our cohort are there in each of the categories we have?
print('count by sex')
print(df['sex'].value_counts())
print('')
print('count by rac')
print(df['rac'].value_counts())
print('')
print('count by rootrace')
print(df['rootrace'].value_counts())
print('')

### Employment and TANF

Now we'll explore how many of our cohort were **employed** before, during, or after Q4 of 2006 (note: we'll further need to use their start_date to say if the job was before or during enrollment in this TANF spell)

In [ ]:
# rather than selecting the cohort in a sub-query, let's use 
# the values from the data frame:
cohort_ssns = df['ssn_hash'].unique()

# reformat the list as a long string of values, this will make it easier to use in the query
cohort_ssns = ','.join(["'"+ssn+"'" for ssn in cohort_ssns])

> This line of code may look complicated, so let's break it down step by step:
>
> 1. __`... for ssn in cohort_ssns ...`__ - Loop through every element `ssn` in the list `cohort_ssns`
> 2. __`"'"+ssn+"'" ...`__ - Return SSN value with single quote
> 3. __`','.join(...)`__ - join all elements of the list with a comma between them
>
> _Additional Note: The formulation `[<action> for <item> in <iterable>]`is known as "list comprehension"._ 

In [ ]:
start_time = time.time()
# start with before and during
# the wage record data starts in 2005 below is only 2 years
query = """
SELECT count(*) recs
FROM il_des_kcmo.il_wage
WHERE year < 2007
    AND ssn IN ({})
""".format(cohort_ssns)

print('there are {} records of "before" or "during" jobs'.format(pd.read_sql(query, conn)['recs'][0]))
print('query completed in {:.2f} seconds'.format(time.time()-start_time))

In [ ]:
start_time = time.time()
# how many after 2007Q4 - we'll only consider the following 2 years
# the wage record data starts in 2005
query = """
SELECT count(*) recs
FROM il_des_kcmo.il_wage
WHERE year IN (2007, 2008)
    AND ssn IN ({})
""".format(cohort_ssns)

print('there are {} records of "after" jobs'.format(pd.read_sql(query, conn)['recs'][0]))
print('query completed in {:.2f} seconds'.format(time.time()-start_time))

In [ ]:
# let's pull the "after" jobs for some further analysis

start_time = time.time()

query = """
SELECT ssn, ein, seinunit, empr_no, wage, year, quarter
FROM il_des_kcmo.il_wage
WHERE year IN (2007, 2008)
    AND ssn IN ({})
""".format(cohort_ssns)
df_jobs = pd.read_sql(query, conn)
print('query completed in {:.2f} seconds'.format(time.time()-start_time))

In [ ]:
df_jobs.info()

In [ ]:
# how many individuals in our cohort had _any_ job in 2007 or 2008?
df_jobs['ssn'].nunique()

In [ ]:
print('{:.1f}% of our cohort had _a_ job in 2007 and/or 2008'.format(\
100.*df_jobs['ssn'].nunique()/df.shape[0]))

## Creating New Measures
- Back to [Table of Contents](#Table-of-Contents)

> **Questions to consider** (we will revisit similar questions of measurement frequently during the program)
0. What problem are we working to solve? How can we measure it?
1. How should we define that an individual "received a job"? For example, definitions could be 
  * Received greater than 0 pay at some point within 1 year
  * Received greater than minimum wage (assuming XyZ) in 6 of 8 quarters after exiting the TANF program
2. How narrowly can you define the unit of analysis? Eg an individual who stopped receiving TANF by...
3. What additional information do we know about these individuals?

**Preliminary Examples**

As the notebooks progress we will dig into different aspects of the above questions, but for now we will show the example of using the `df` dataframe as our study cohort and the `df_jobs` dataframe to create a few example ways to define whether each individual received a job after leaving TANF.

In [ ]:
# simple example of getting "any" job
df['ssn_hash'].isin(df_jobs['ssn'].unique()).value_counts()

In [ ]:
# simple example of getting "any" job - add as column to `df` DataFrame
df['emp_any_job'] = df['ssn_hash'].isin(df_jobs['ssn'].unique())

In [ ]:
# at least one quarter's earnings are over "full-time minimum wage" value of $2,730 in 2007
df_jobs[df_jobs['wage']>=2730]['ssn'].nunique()

In [ ]:
df['emp_1qtr_overMin'] = df['ssn_hash'].isin(df_jobs[df_jobs['wage']>=2730]['ssn'].unique())
df['emp_1qtr_overMin'].value_counts(normalize=True)

In [ ]:
# at least 4 quarters' earnings over "full-time minimum wage" value of $2,730 in 2007
df_jobs[df_jobs['wage']>=2730].groupby('ssn')['wage'].count().sort_values(ascending=False).head(10)

In [ ]:
# check records for specific ssn value
ssn_val = '...'
df_jobs.query("ssn == '{}'".format(ssn_val))

In [ ]:
# create temporary dataframe
temp_df = df_jobs[df_jobs['wage']>=2730].groupby('ssn')['wage'].count().reset_index().rename(columns={'wage':'count'})

# add outcome measure to df
df['emp_2qrts_overMin'] = df['ssn_hash'].isin(temp_df[temp_df['count']>=2]['ssn'])
df['emp_2qrts_overMin'].value_counts(normalize=True)

In [ ]:
df.columns

In [ ]:
outcome_list = ['emp_any_job', 'emp_1qtr_overMin','emp_2qrts_overMin']
df[outcome_list].sum() # number of True for each outcome metric

__Separate example: Replicating the QWI Statistics__

The [QWI Statistics](../notebooks_additional/03_2_QWI_Statistics.ipynb) notebook demonstrates another example of feature creation: the Quarterly Workforce Indicators Census framework using IL wage records.